In [1]:
# Libraries
import json
import math
import random
import pandas as pd

# Matching libaries
import numpy as np

# Plot libraries
import folium
import seaborn as sns
import matplotlib.pyplot as plt


from scripts.data_loader import DataLoader
from scripts.data_formatter import DataProcessor as dp
from scripts.matching import hungarian_method
from scripts.plotter import Plotter as pl

from datetime import datetime
now = datetime.now()

current_time = now.strftime("%d%m_%H%M%S")

# TO-DOS
Sort data for longest axis is length and shortest is height. Since the ship is always a rectangle and not a cube.

In [ ]:
# Define date and time filter
date_key = '03-11-2022'
# Define paths
base_path = "C:\\Users\\abelt\\OneDrive\\Desktop\\Kandidat\\"
## File names ##
# AIS
ais_files = {
    '02-11-2022': 'ais\\ais_110215.csv',
    '03-11-2022': 'ais\\ais_110315.csv',
    '05-11-2022': 'ais\\ais_1105.csv'
}
# SAR
sar_files = {
    '02-11-2022': 'sar\\Sentinel_1_detection_20221102T1519.json',
    '03-11-2022': 'sar\\Sentinel_1_detection_20221103T154515.json',
    '05-11-2022': 'sar\\Sentinel_1_detection_20221105T162459.json'
}
# Norsat
norsat_files = {
    '02-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-02T151459Z.json',
    '03-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-03T152759Z.json',
    '05-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-05T155259Z.json'
}

data_loader = DataLoader(base_path = base_path, ais_files = ais_files, sar_files = sar_files, norsat_files = norsat_files, date_key = date_key)
ais_data, sar_data, norsat_data = data_loader.load_data()


# print('Interpolation for ais')
# print('SAR')
# ais_sar_interpol, sar_missing = dp.ais_interpolate_mmsi_points(ais_data = ais_data['tm_ais_sar'], date_key = date_key, interpolation_time_col = sar_data['sar_landmasked']['TimeStamp'])
# print('Norsat-3')
# ais_norsat_interpol, norsat_missing = dp.ais_interpolate_mmsi_points(ais_data = ais_data['tm_ais_norsat'], date_key = date_key, interpolation_time_col = norsat_data[date_key]['TimeStamp'])

In [4]:
# MATCHING

# Hungarian # 
########## Haversine ##########

########## Nearest Neighbor Matching ##########

def match_ships_to_norsat_nearest_neighbor(ship_dict, norsat_data):
    all_matches = []
    
    # Iterate over each ship in the dictionary
    for mmsi, coords in ship_dict.items():
        ship_lat, ship_lon = coords['y'], coords['x']
        nearest = None
        min_distance = float('inf')
        
        # Iterate over each row in the norsat dataframe
        for _, norsat_row in norsat_data.iterrows():
            norsat_lat, norsat_lon, norsat_idx = norsat_row['latitude'], norsat_row['longitude'], norsat_row['norsat_id']
            
            # Calculate haversine distance
            distance = haversine(ship_lat, ship_lon, norsat_lat, norsat_lon)
            
            # Check if this NORSAT point is closer than the current nearest
            if distance < min_distance:
                min_distance = distance
                nearest = {
                    'mmsi': mmsi,
                    'ais_lat': ship_lat,
                    'ais_lon': ship_lon,
                    'norsat_lat': norsat_lat,
                    'norsat_lon': norsat_lon,
                    'norsat_distance_km': distance,
                    'norsat_id': norsat_idx  # Include the index of the NORSAT row
                }
        
        # If a nearest match was found, add it to the results
        if nearest :
            all_matches.append(nearest )
    
    return pd.DataFrame(all_matches)

########## SAR ##########
# Function to find two nearest matches for each AIS point
def two_nearest_neighbors(ais_data, sar_data):
    matches = []

    # Iterate over each AIS point in the dictionary
    for mmsi, coords in ais_data.items():
        ais_lat, ais_lon = coords['y'], coords['x']
        
        # Create a list to store distances to all SAR points
        sar_distances = []
        
        # Calculate distance to all SAR points and store them
        for _, sar_row in sar_data.iterrows():
            sar_lat, sar_lon, sar_idx = sar_row['latitude'], sar_row['longitude'], sar_row['sar_id']
            distance = haversine(ais_lat, ais_lon, sar_lat, sar_lon)
            sar_distances.append((distance, sar_idx, sar_lat, sar_lon))
        
        # Sort distances to find the two smallest ones
        sar_distances.sort(key=lambda x: x[0])  # Sort by distance (first element of tuple)
        
        nearest = sar_distances[0]
        second_nearest = sar_distances[1]
            
            # Append both matches to the results
        matches.append({
                'mmsi': mmsi,
                'ais_lat': ais_lat,
                'ais_lon': ais_lon,
                'sar_id' : nearest[1],
                'sar_lat': nearest[2],
                'sar_lon': nearest[3],
                'sar_distance_km': nearest[0]
        })

    return pd.DataFrame(matches)

In [5]:
#### Plots ####

# map_object = pl.norsat_plot_uncertainty_ellipses(norsat_data = norsat_data, date_key = date_key)
# map_object.save(f'./images/ellipses_map_{date_key}.html')
# map_object  # Display the map

#object_map = pl.unified_plot(ais_mmsi = time_matching_ais_sar, sar_data = SAR_on_sea, norsat_data = None, date_key = date_key)
#object_map.save(f'./images/SAR_AIS_{date_key}_{current_time}.html')

#object_map1 = pl.unified_plot(ais_mmsi = None, sar_data = None, norsat_data = norsat_data, interpolated_ais = ais_norsat_interpol, date_key = date_key)
#object_map1.save(f'./images/Norsat_AIS_{date_key}_{current_time}.html')

def generate_random_color():
        r = lambda: random.randint(0, 255)
        return '#{:02x}{:02x}{:02x}'.format(r(), r(), r())

In [6]:
#### Get unique matches ###
def get_unique_shortest_distance_norsat(df):
    # Step 1: Group by 'mmsi' and 'sar_id', and select the row with the shortest distance for each group
    shortest_distance_df = df.loc[df.groupby(['mmsi', 'norsat_id'])['norsat_distance_km'].idxmin()].reset_index(drop=True)
    
    # Step 2: Ensure unique 'mmsi' and 'sar_id' by filtering out duplicates
    # Sort by distance so we prioritize the smallest distance when removing duplicates
    sorted_df = shortest_distance_df.sort_values(by='norsat_distance_km')

    # Drop rows where 'mmsi' or 'sar_id' appear more than once
    return sorted_df.drop_duplicates(subset='mmsi', keep='first').drop_duplicates(subset='norsat_id', keep='first')

def get_unique_shortest_distance_sar(df):
    # Step 1: Group by 'mmsi' and 'sar_id', and select the row with the shortest distance for each group
    shortest_distance_df = df.loc[df.groupby(['mmsi', 'sar_id'])['sar_distance_km'].idxmin()].reset_index(drop=True)
    
    # Step 2: Ensure unique 'mmsi' and 'sar_id' by filtering out duplicates
    # Sort by distance so we prioritize the smallest distance when removing duplicates
    sorted_df = shortest_distance_df.sort_values(by='sar_distance_km')

    # Drop rows where 'mmsi' or 'sar_id' appear more than once
    return sorted_df.drop_duplicates(subset='mmsi', keep='first').drop_duplicates(subset='sar_id', keep='first')

In [8]:
hung = hungarian_method(ais_sar_interpol, sar_data['sar_landmasked'])
matched_ships_df = match_ships_to_norsat_nearest_neighbor(ship_dict = ais_norsat_interpol, norsat_data = norsat_data[date_key])

sar_data['sar_landmasked'].loc[:, 'sar_id'] = range(1, len(sar_data['sar_landmasked']) + 1)
norsat_data[date_key].loc[:,'norsat_id'] = range(1, len(norsat_data[date_key]) + 1)

In [ ]:
# Match:
norsat_matched_ships_df = match_ships_to_norsat_nearest_neighbor(ship_dict = ais_norsat_interpol, norsat_data = norsat_data[date_key])
sar_matched_ships_df = two_nearest_neighbors(ais_sar_interpol, sar_data['sar_landmasked'])

# Get only uniques:
unique_norsat_df = get_unique_shortest_distance_norsat(norsat_matched_ships_df)
unique_sar_df = get_unique_shortest_distance_sar(sar_matched_ships_df)

# Threshold for distance
unique_sar_df_tresholded = unique_sar_df[unique_sar_df['sar_distance_km'] <= 50.25]
unique_norsat_df_tresholded = unique_norsat_df[unique_norsat_df['norsat_distance_km']<= 50.25]

# Triple matchings
triple_match_df = unique_norsat_df_tresholded.merge(unique_sar_df_tresholded, on=['mmsi'])

print(min(norsat_data[date_key]['latitude']),max(norsat_data[date_key]['latitude']))
print(min(norsat_data[date_key]['longitude']),max(norsat_data[date_key]['longitude']))


In [18]:
# lable lat and lon #
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

def add_lat_lon_labels(ax, draw_labels=True, label_size=12):
    """
    Adds latitude and longitude labels to a Cartopy map plot.
    
    Args:
        ax (GeoAxes): The Cartopy GeoAxes object to which labels are added.
        draw_labels (bool): Whether to draw the latitude and longitude labels.
        label_size (int): Font size of the labels.
    
    Returns:
        None
    """
    # Add gridlines to the map
    gl = ax.gridlines(draw_labels=draw_labels, crs=ccrs.PlateCarree(), linestyle='--', alpha=0.5)

    # Customize longitude labels (bottom and top of the map)
    gl.xlabels_top = False  # Disable labels at the top
    gl.xlabels_bottom = True  # Enable labels at the bottom
    gl.xformatter = cticker.LongitudeFormatter()  # Longitude labels
    gl.xlabel_style = {'size': label_size, 'rotation': 0}

    # Customize latitude labels (left and right of the map)
    gl.ylabels_right = False  # Disable labels on the right
    gl.ylabels_left = True  # Enable labels on the left
    gl.yformatter = cticker.LatitudeFormatter()  # Latitude labels
    gl.ylabel_style = {'size': label_size, 'rotation': 0}


In [23]:
# Wrong point #
def plot_wrong_point(df):
    """
    Plots matches between AIS and either SAR or Norsat data based on the DataFrame input.

    Args:
        df (DataFrame): DataFrame containing either SAR or Norsat data with latitude and longitude columns.

    Returns:
        None: Displays the map.
    """

    # Determine if the DataFrame is for SAR or Norsat based on column names
    if 'sar_id' in df.columns:
        lat_col = 'sar_lat'
        lon_col = 'sar_lon'
        target_type = 'SAR'
        colors = 'red'
        mark = 'd'

    elif 'norsat_id' in df.columns:
        lat_col = 'norsat_lat'
        lon_col = 'norsat_lon'
        target_type = 'Norsat'
        colors = 'green'
        mark = 'x'
    else:
        raise ValueError("DataFrame doesn't contain SAR or Norsat columns")

    num_matches = len(df)

    # Define the extent for the map around 70° 12' 50" N, 23° 12' 0" E
    center_lat = 70.2139  # Approximate latitude of the center location
    center_lon = 23.2000  # Approximate longitude of the center location
    lat_buffer = .30  # Increase buffer to zoom out more
    lon_buffer = .40  # Increase buffer to zoom out more

    # Create a new figure with the PlateCarree projection (appropriate for latitude/longitude data)
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # Set the extent of the map around the specific location
    ax.set_extent([center_lon - lon_buffer, center_lon + lon_buffer, center_lat - lat_buffer, center_lat + lat_buffer], 
                  crs=ccrs.PlateCarree())

    # Add latitude and longitude labels
    add_lat_lon_labels(ax)

    # Add features to the map
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, color='lightgray')
    ax.add_feature(cfeature.OCEAN, color='lightblue')
    ax.add_feature(cfeature.LAKES, color='lightblue')
    ax.add_feature(cfeature.RIVERS)
    
    size = 5
    # Plot AIS points
    ax.scatter(df['ais_lon'], df['ais_lat'], 
               color='blue', label='AIS', zorder=5, transform=ccrs.PlateCarree(), s=size)

    # Plot target points (SAR or Norsat)
    ax.scatter(df[lon_col], df[lat_col], 
               marker=mark, color=colors, label=target_type, zorder=5, transform=ccrs.PlateCarree(), s=size)

    # Plot lines between AIS and target (SAR or Norsat)
    for _, row in df.iterrows():
        ax.plot([row['ais_lon'], row[lon_col]], 
                [row['ais_lat'], row[lat_col]], 
                color='black', linestyle='--', alpha=0.6, transform=ccrs.PlateCarree())

    # Add legend
    plt.legend()
    plt.title(f'Matches Between AIS and {target_type}')#, matches: {num_matches}')
    plt.show()


In [19]:
# single match #
def plot_matches(df):
    """
    Plots matches between AIS and either SAR or Norsat data based on the DataFrame input.

    Args:
        df (DataFrame): DataFrame containing either SAR or Norsat data with latitude and longitude columns.

    Returns:
        None: Displays the map.
    """

    # Determine if the DataFrame is for SAR or Norsat based on column names
    if 'sar_id' in df.columns:
        lat_col = 'sar_lat'
        lon_col = 'sar_lon'
        target_type = 'SAR'
        colors = 'red'
        mark = 'd'

    elif 'norsat_id' in df.columns:
        lat_col = 'norsat_lat'
        lon_col = 'norsat_lon'
        target_type = 'Norsat'
        colors = 'green'
        mark = 'x'
    else:
        raise ValueError("DataFrame doesn't contain SAR or Norsat columns")

    num_matches = len(df)

    # Calculate the extent dynamically based on latitude and longitude columns
    min_lon = df[['ais_lon', lon_col]].min().min()
    max_lon = df[['ais_lon', lon_col]].max().max()
    min_lat = df[['ais_lat', lat_col]].min().min()
    max_lat = df[['ais_lat', lat_col]].max().max()

    # Create a new figure with the European Albers Equal Area projection
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.AlbersEqualArea(central_longitude=10, central_latitude=52, 
                                                                   standard_parallels=(43, 62)))

    # Set the extent of the map dynamically with a small buffer for better visibility
    ax.set_extent([min_lon - 0.5, max_lon + 0.5, min_lat - 0.5, max_lat + 0.5], crs=ccrs.PlateCarree())

    # Add latitude and longitude labels
    add_lat_lon_labels(ax)

    # Add features to the map
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, color='lightgray')
    ax.add_feature(cfeature.OCEAN, color='lightblue')
    ax.add_feature(cfeature.LAKES, color='lightblue')
    ax.add_feature(cfeature.RIVERS)
    
    size=5
    # Plot AIS points
    ax.scatter(df['ais_lon'], df['ais_lat'], 
               color='blue', label='AIS', zorder=5, transform=ccrs.PlateCarree(), s = size)

    # Plot target points (SAR or Norsat)
    ax.scatter(df[lon_col], df[lat_col], 
               marker = mark, color=colors, label=target_type, zorder=5, transform=ccrs.PlateCarree(), s = size)

    # Plot lines between AIS and target (SAR or Norsat)
    for _, row in df.iterrows():
        ax.plot([row['ais_lon'], row[lon_col]], 
                [row['ais_lat'], row[lat_col]], 
                color='black', linestyle='--', alpha=0.6, transform=ccrs.PlateCarree())

    

    plt.legend()
    plt.title(f'Matches Between AIS and {target_type}, matches: {num_matches}')
    plt.show()


In [20]:
# Triple match # 
def plot_triple_matches_on_cartopy(df):
    """
    Visualizes the positions of vessels from AIS data, Norsat data, and SAR data on a Cartopy map. 
    The function creates markers for each vessel's location and connects them with lines to illustrate relationships.

    Args:
        df (DataFrame): A pandas DataFrame containing vessel data with latitude and longitude columns for AIS, Norsat, and SAR.

    Returns:
        None: Displays the map with vessel locations and connections.
    """

    num_matches = len(df)

    # Calculate the extent dynamically based on latitude and longitude columns
    min_lon = df[['ais_lon_x', 'ais_lon_y', 'norsat_lon', 'sar_lon']].min().min()
    max_lon = df[['ais_lon_x', 'ais_lon_y', 'norsat_lon', 'sar_lon']].max().max()
    min_lat = df[['ais_lat_x', 'ais_lat_y', 'norsat_lat', 'sar_lat']].min().min()
    max_lat = df[['ais_lat_x', 'ais_lat_y', 'norsat_lat', 'sar_lat']].max().max()

    # Create a new figure with the European Albers Equal Area projection
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.AlbersEqualArea(central_longitude=10, central_latitude=52, 
                                                                   standard_parallels=(43, 62)))

    # Set the extent of the map dynamically with a small buffer for better visibility
    ax.set_extent([min_lon - 0.5, max_lon + 0.5, min_lat - 0.5, max_lat + 0.5], crs=ccrs.PlateCarree())
    # Add latitude and longitude labels
    add_lat_lon_labels(ax)
    
    # Add coastlines and other geographical features
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, color='lightgray')
    ax.add_feature(cfeature.OCEAN, color='lightblue')
    ax.add_feature(cfeature.LAKES, color='lightblue')
    ax.add_feature(cfeature.RIVERS)

    # Iterate over the rows of the dataframe to plot markers and lines
    for _, row in df.iterrows():
        # Plot AIS X location (ais_lat_x, ais_lon_x)
        ax.plot(row['ais_lon_x'], row['ais_lat_x'], marker='o', color='blue', label='AIS X' if _ == 0 else "", transform=ccrs.PlateCarree())

        # Plot AIS Y location (ais_lat_y, ais_lon_y)
        ax.plot(row['ais_lon_y'], row['ais_lat_y'], marker='o', color='blue', label='AIS Y' if _ == 0 else "", transform=ccrs.PlateCarree())

        # Plot Norsat location (norsat_lat, norsat_lon)
        ax.plot(row['norsat_lon'], row['norsat_lat'], marker='d', color='red', label='Norsat' if _ == 0 else "", transform=ccrs.PlateCarree())

        # Plot SAR location (sar_lat, sar_lon)
        ax.plot(row['sar_lon'], row['sar_lat'], marker='X', color='green', label='SAR' if _ == 0 else "", transform=ccrs.PlateCarree())

        # Draw lines between AIS X and Norsat
        ax.plot([row['ais_lon_x'], row['norsat_lon']], [row['ais_lat_x'], row['norsat_lat']], color='blue', linestyle='--', transform=ccrs.PlateCarree())
        # Draw lines between AIS Y and SAR
        ax.plot([row['ais_lon_y'], row['sar_lon']], [row['ais_lat_y'], row['sar_lat']], color='blue', linestyle='--', transform=ccrs.PlateCarree())
        # Draw lines connecting AIS X and AIS Y
        ax.plot([row['ais_lon_x'], row['ais_lon_y']], [row['ais_lat_x'], row['ais_lat_y']], color='orange', linestyle='--', transform=ccrs.PlateCarree())

    # Add legend to the map
    plt.legend(loc='upper right')
    plt.title(f'Triple matching, matches: {num_matches}')
    # Display the map
    plt.show()


In [25]:
# Good match #
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def plot_good_matches(df):
    """
    Visualizes the good matches between AIS data, Norsat data, and SAR data on a Cartopy map.

    Args:
        df (DataFrame): A pandas DataFrame containing vessel data with latitude and longitude columns for AIS, Norsat, and SAR.

    Returns:
        None: Displays the map with vessel locations and connections.
    """

    num_matches = len(df)

    # Define the extent for the map around 69°46'28.8"N, 19°10'35.7"E
    center_lat = 69.7747  # Latitude of the new center location
    center_lon = 19.1766  # Longitude of the new center location
    lat_buffer = 0.7  # Buffer to control zoom level (adjust for desired zoom)
    lon_buffer = 0.7  # Buffer to control zoom level (adjust for desired zoom)

    # Create a new figure with the PlateCarree projection
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    # Set the extent of the map around the new specific location
    ax.set_extent([center_lon - lon_buffer, center_lon + lon_buffer, center_lat - lat_buffer, center_lat + lat_buffer], 
                  crs=ccrs.PlateCarree())

    # Add latitude and longitude labels
    add_lat_lon_labels(ax)

    # Add coastlines and other geographical features
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, color='lightgray')
    ax.add_feature(cfeature.OCEAN, color='lightblue')
    ax.add_feature(cfeature.LAKES, color='lightblue')
    ax.add_feature(cfeature.RIVERS)

    # Iterate over the rows of the dataframe to plot markers and lines
    for _, row in df.iterrows():
        # Plot AIS X location (ais_lat_x, ais_lon_x)
        ax.plot(row['ais_lon_x'], row['ais_lat_x'], marker='o', color='blue', label='AIS (RF)' if _ == 0 else "", transform=ccrs.PlateCarree())

        # Plot AIS Y location (ais_lat_y, ais_lon_y)
        ax.plot(row['ais_lon_y'], row['ais_lat_y'], marker='o', color='blue', label='AIS (SAR)' if _ == 0 else "", transform=ccrs.PlateCarree())

        # Plot Norsat location (norsat_lat, norsat_lon)
        ax.plot(row['norsat_lon'], row['norsat_lat'], marker='X', color='green', label='Norsat' if _ == 0 else "", transform=ccrs.PlateCarree())

        # Plot SAR location (sar_lat, sar_lon)
        ax.plot(row['sar_lon'], row['sar_lat'], marker='d', color='red', label='SAR' if _ == 0 else "", transform=ccrs.PlateCarree())

        # Draw lines between AIS X and Norsat
        ax.plot([row['ais_lon_x'], row['norsat_lon']], [row['ais_lat_x'], row['norsat_lat']], color='blue', linestyle='--', transform=ccrs.PlateCarree())
        # Draw lines between AIS Y and SAR
        ax.plot([row['ais_lon_y'], row['sar_lon']], [row['ais_lat_y'], row['sar_lat']], color='blue', linestyle='--', transform=ccrs.PlateCarree())
        # Draw lines connecting AIS X and AIS Y
        ax.plot([row['ais_lon_x'], row['ais_lon_y']], [row['ais_lat_x'], row['ais_lat_y']], color='orange', linestyle='--', transform=ccrs.PlateCarree())

    # Add legend to the map
    plt.legend(loc='upper right')
    plt.title(f'Triple matches, AIS, SAR and RF')
    # Display the map
    plt.show()

In [ ]:
# histogram with PDF #
import seaborn as sns

def calculate_rms_distance(df, distance_col):
    """
    Calculate the RMS for a given distance column in the DataFrame.
    """
    return np.sqrt(np.mean(df[distance_col] ** 2))

def plot_distance_histogram_with_pdf(df, distance_col, title):
    """
    Create a histogram with PDF (kde) of the distance between targets and annotate with RMS.
    
    Parameters:
    - df: DataFrame containing the distance column
    - distance_col: Column name for distances
    - title: Title of the plot
    """
    # Calculate RMS for the distance
    rms_distance = calculate_rms_distance(df, distance_col)

    # Number of bins equal to the number of data points to ensure all distances are visible
    num_bins = len(df[distance_col])

    # Create a histogram with count on y-axis and PDF overlay
    plt.figure(figsize=(10, 6))
    sns.histplot(df[distance_col], kde=True, color='blue', bins=num_bins, stat="count", label='Distance with PDF')

    # Annotate the RMS on the plot
    plt.axvline(rms_distance, color='red', linestyle='--', label=f'RMS = {rms_distance:.4f}')
    plt.text(rms_distance, plt.ylim()[1]*0.8, f'RMS: {rms_distance:.4f}', color='red', fontsize=12, ha='center')

    # Titles and labels
    plt.title(title)
    plt.xlabel('Distance (km)')
    plt.ylabel('Count')  # Set y-axis label to 'Count'
    plt.legend()
    plt.grid(True)
    plt.show()

In [56]:
#plot_triple_matches_on_cartopy(triple_match_df)

#plot_matches(unique_sar_df_tresholded)
#plot_matches(unique_norsat_df_tresholded)

In [22]:
# Folium
folium_map_sar = pl.plot_matches_on_folium(unique_sar_df_tresholded)
folium_map_norsat = pl.plot_matches_on_folium(unique_norsat_df_tresholded) 
folium_map = pl.plot_triple_matches_on_folium(triple_match_df)

folium_map_sar.save(f"./images/matches_map_sar{current_time}.html")
folium_map_norsat.save(f"./images/matches_map_norsat{current_time}.html")

folium_map.save(f"./images/triple_matches_map_5_{current_time}.html")

In [55]:
#plot_wrong_point(unique_sar_df_tresholded)

In [54]:
#plot_good_matches(triple_match_df)

In [ ]:
# Plot for SAR vs AIS distance
plot_distance_histogram_with_pdf(unique_sar_df_tresholded, 'sar_distance_km', 'Histogram of SAR vs AIS Distance with PDF and RMS')

# Plot for Norsat vs AIS distance
plot_distance_histogram_with_pdf(unique_norsat_df_tresholded, 'norsat_distance_km', 'Histogram of Norsat vs AIS Distance with PDF and RMS')

In [ ]:
# Plot for SAR vs AIS distance
plot_distance_histogram_with_pdf(unique_sar_df_tresholded, 'sar_distance_km', 'Histogram of SAR vs AIS Distance with PDF and RMS')

# Plot for Norsat vs AIS distance
plot_distance_histogram_with_pdf(unique_norsat_df_tresholded, 'norsat_distance_km', 'Histogram of Norsat vs AIS Distance with PDF and RMS')

Beskriv med "egen fornuft" fejlkilder, for at begynder og overveje hvordan de kan mitigeres, fjernes eller accepteres.

Real sar opløsning 22 meter, men oversampler for til 10x10